## テーマ
  投資対象を絞り込む用途に用いる。
  そのため、具体的な投資対象を一意に絞り込む用途は想定していない。

### 用途目的
  投資対象として有望なものを導出する。
　１．一年分のチャート等を読み込ませ今後の動向を分類させる。（急騰、暴落、停滞、微増、微減）など

In [1]:
#from chainer import cuda
from chainer import FunctionSet
from chainer import Variable
from chainer import optimizers
import chainer.functions  as F
from sklearn.datasets import fetch_mldata
import numpy as np
import model_util
models=model_util.ModelUtil()

In [2]:
def set_data(target):
    if 'mnist' == target:
        # MNISTの手書き数字データのダウンロード
        # #HOME/scikit_learn_data/mldata/mnist-original.mat にキャッシュされる
        mnist = fetch_mldata('MNIST original')
        # mnist.data : 70,000件の784次元ベクトルデータ
        # だいたい一つの数字につき7000件前後　手書き数字データ
        mnist.data   = mnist.data.astype(np.float32)
        mnist.data  /= 255     # 0-1のデータに変換

        # mnist.target : 正解データ（教師データ）
        mnist.target = mnist.target.astype(np.int32)

        # 学習用データを N個、検証用データを残りの個数と設定
        N = 56000
    #     print(len(mnist.data))
        x_train, x_test = np.split(mnist.data,   [N])
        y_train, y_test = np.split(mnist.target, [N])
    else:
        print 'hogehoge'

    return x_train, x_test, y_train, y_test, N

In [3]:
# ニューラルネットの構造
def forward(x_data, train=True):
    # 活性化関数を選択
    # sigmoid関数
#     h1 = F.sigmoid(model.l1(x))
#     h2 = F.sigmoid(model.l2(h1))

    # dropoutを追加
    # ReLU関数
    h1 = F.dropout(F.relu(model.l1(x)),  train=train)
    h2 = F.dropout(F.relu(model.l2(h1)), train=train)
    y  = model.l3(h2)

    return y

In [12]:
def train(x_batch, y_batch, train=True):
    # 型を変換
    x = Variable(x_batch.astype(np.float32), volatile=False) 
    t = Variable(y_batch.astype(np.int32), volatile=False)
    p = forward(x, train)

    # 勾配を初期化
    optimizer.zero_grads()
    print("x: ndim {0} data {1}".format(x.ndim,len(x.data)))
    print("p: ndim {0} data {1}".format(p.ndim,len(p.data)))
    print("p_class: {}".format(p.data.argmax()))

    # 順伝播させて誤差と精度を算出
    # 多クラス分類なので誤差関数としてソフトマックス関数の
    # 交差エントロピー関数を用いて、誤差を導出
    loss = F.softmax_cross_entropy(p, t)
#         loss = F.mean_squared_error(p, t)
    acc = F.accuracy(p, t)

    # 誤差逆伝播で勾配を計算
    loss.backward()
    optimizer.update()
    return loss, acc

In [4]:
# データ設定
target='mnist'
x_train, x_test, y_train, y_test, N = set_data(target)
N_test = y_test.size
    
# Prepare multi-layer perceptron model
# 多層パーセプトロンモデルの設定
# 入力 784次元、出力 10次元
# 中間層のunit数
# 一層目
n_units = 200
# 二層目
n2_units = 100

model = FunctionSet(l1=F.Linear(784, n_units),
                    l2=F.Linear(n_units, n2_units),
                    l3=F.Linear(n2_units, 10))

# Setup optimizer
optimizer = optimizers.Adam()
optimizer.setup(model)

train_loss = []
train_acc  = []
test_loss = []
test_acc  = []

l1_W = []
l2_W = []
l3_W = []


In [14]:
print len(y_test)
print N_test

14000
14000


In [13]:

# 確率的勾配降下法で学習させる際の１回分のバッチサイズ
batchsize = 100

# 学習の繰り返し回数
# n_epoch   = 20
n_epoch   = 1
# Learning loop
for epoch in xrange(1, n_epoch+1):
    print 'epoch', epoch

    # training
    # N個の順番をランダムに並び替える
    perm = np.random.permutation(N)
    sum_accuracy = 0
    sum_loss = 0

    # 0〜Nまでのデータをバッチサイズごとに使って学習
    for i in xrange(0, N, batchsize):
        x_batch = x_train[perm[i:i+batchsize]]
        y_batch = y_train[perm[i:i+batchsize]]
        loss, acc = train(x_batch, y_batch)

#         sum_loss     += float(cuda.to_cpu(loss.data)) * batchsize
#         sum_accuracy += float(cuda.to_cpu(acc.data)) * batchsize
        sum_loss     += float(loss.data) * batchsize
        sum_accuracy += float(acc.data) * batchsize
        sys.exit()
    # 訓練データの誤差と、正解精度を表示
    #print 'train mean loss={}, accuracy={}'.format(sum_loss / N, sum_accuracy / N)

    train_loss.append(sum_loss / N)
    train_acc.append(sum_accuracy / N)

    # evaluation
    # テストデータで誤差と、正解精度を算出し汎化性能を確認
    sum_accuracy = 0
    sum_loss     = 0
    for i in xrange(0, N_test, batchsize):
        x_batch = x_test[i:i+batchsize]
        y_batch = y_test[i:i+batchsize]

        # 型を変換
        x = Variable(x_batch) 
        t = Variable(y_batch)
        loss, acc = train(x_batch, y_batch, train=False)

#         sum_loss     += float(cuda.to_cpu(loss.data)) * batchsize
#         sum_accuracy += float(cuda.to_cpu(acc.data)) * batchsize
        sum_loss     += float(loss.data) * batchsize
        sum_accuracy += float(acc.data) * batchsize

    # テストデータでの誤差と、正解精度を表示
    #print 'test  mean loss={}, accuracy={}'.format(sum_loss / N_test, sum_accuracy / N_test)
    test_loss.append(sum_loss / N_test)
    test_acc.append(sum_accuracy / N_test)

    # 学習したパラメーターを保存
    l1_W.append(model.l1.W)
    l2_W.append(model.l2.W)
    l3_W.append(model.l3.W)

models.set_model_pkl('{0}.pkl'.format(target))
models.set_model_name('{0}.model'.format(target))
models.set_optimizer_name('{0}.state'.format(target))
models.dump_model(model)
models.dump_model_and_optimizer(model, optimizer)
    
# 精度と誤差をグラフ描画
plt.figure(figsize=(8,6))
plt.plot(range(len(train_acc)), train_acc)
plt.plot(range(len(test_acc)), test_acc)
plt.legend(["train_acc","test_acc"],loc=4)
plt.title("Accuracy of digit recognition.")
plt.plot()

epoch 1
x: ndim 2 data 100
p: ndim 2 data 100
p_class: 199


NameError: name 'sys' is not defined